In [12]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json

In [13]:
playtest5_data = pd.read_csv("2018-11-14_Playtest/anonymized_playtest5_data.csv", sep=";")
playtest6_data = pd.read_csv("2019-01-07_Playtest/anonymized_playtest6_data.csv", sep=";")
playtest7_data = pd.read_csv("2019-01-31_Playtest/anonymized_playtest7_data.csv", sep=";")

In [15]:
all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])

In [16]:
all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])

In [17]:
all_playtest_data = all_playtest_data.sort_values('time')

In [18]:
data_by_user = all_playtest_data.groupby('session_id').agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'n_events',
                                                                                              'type':'n_different_events'})

In [19]:
data_by_user['active_time'] = np.nan
data_by_user['avg_dt'] = np.nan
data_by_user['std_dt'] = np.nan

In [20]:
data_by_user.index = data_by_user['session_id'].values

In [21]:
for user in data_by_user['session_id'].unique():
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
   
    # Computing active time
    previousEvent = None
    theresHoldActivity = 15 # np.percentile(allDifferences, 98) is 10 seconds
    activeTime = []
    
    for enum, event in user_events.iterrows():
        
        # If it is the first event
        if(previousEvent is None):
            previousEvent = event
            continue
        
        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
        if(~(delta_seconds > theresHoldActivity)):
            activeTime.append(delta_seconds)
        
        previousEvent = event
        
    data_by_user.at[user, 'active_time'] = round(np.sum(activeTime)/60,2)
    data_by_user.at[user, 'avg_dt'] = round(np.mean(activeTime)/60,2)
    data_by_user.at[user, 'std_dt'] = round(np.std(activeTime)/60,2)

In [22]:
data_by_user

,session_id,n_events,n_different_events,active_time,avg_dt,std_dt
08lyf80qem23ddyruo38pm1h1wpp3q6l-Playtest5-First,08lyf80qem23ddyruo38pm1h1wpp3q6l-Playtest5-First,89,18,4.16,0.05,0.13
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-First,651,20,40.28,0.06,0.13
151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-Second,151x2d2unko7w68215pb1fs1azmw36a1-Playtest5-Second,688,19,35.65,0.05,0.18
3h0qqdfi1wb2ypbrhlocmugg6dzhbv5x-Playtest5-First,3h0qqdfi1wb2ypbrhlocmugg6dzhbv5x-Playtest5-First,30,7,0.25,0.01,0.00
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,906,22,94.25,0.10,1.45
4kocnrzlajnjs74jvmmfqc2w4cx1yupv-Playtest6,4kocnrzlajnjs74jvmmfqc2w4cx1yupv-Playtest6,1151,24,110.74,0.10,1.27
6nzz410e7phtq3a16qp5wfsh2phavp2s-Playtest5-First,6nzz410e7phtq3a16qp5wfsh2phavp2s-Playtest5-First,26,12,3.05,0.12,0.44
7cqmyr6a3e1nbghad9a1o4yhlcfk9ojf-Playtest5-First,7cqmyr6a3e1nbghad9a1o4yhlcfk9ojf-Playtest5-First,802,20,38.52,0.05,0.06
7kd277wea4qj9han9ktqgvtynze7vw16-Playtest6,7kd277wea4qj9han9ktqgvtynze7vw16-Playtest6,1154,21,84.38,0.07,1.26
9spkpp5ep82pw8vbdvii0ynjknb2ckqn-Playtest5-First,9spkpp5ep82pw8vbdvii0ynjknb2ckqn-Playtest5-First,765,19,9.85,0.01,0.11
